# 1D sub-pillar modeling: Transformers

Runs `huggingface-multihead` on SageMaker to train transformer-based classifiers.

* 1D pillars and subpillars preprocessing
* Multihead (per pillar) transformer sequence classification
* SageMaker training jobs
* Macro precision, recall, fscore evaluation
* MLFlow tracking


In [ ]:
import os
import sys

sys.path.append(os.path.abspath(os.getcwd() + "../../../../"))

In [ ]:
import sagemaker
from deep.constants import DEV_BUCKET
from deep.utils import formatted_time

sess = sagemaker.Session(default_bucket=DEV_BUCKET.name)
job_name = f"1D-test-{formatted_time()}" 

## Data

In [ ]:
import pandas as pd

train_df = pd.read_csv("../../../data/frameworks_data/data_v0.5/data_v0.5_train.csv")
val_df = pd.read_csv("../../../data/frameworks_data/data_v0.5/data_v0.5_val.csv")

sample = False  # To make the computations faster, sample = True.

if sample:
    train_df = train_df.sample(n=1000)
    val_df = val_df.sample(n=1000)

In [ ]:
input_path = DEV_BUCKET / 'training' / 'input_data' / job_name  # Do not change this

train_path = str(input_path / 'train_df.pickle')
val_path = str(input_path / 'test_df.pickle')

train_df.to_pickle(train_path, protocol=4)  # protocol 4 is necessary, since SageMaker uses python 3.6
val_df.to_pickle(val_path, protocol=4)

## Sagemaker

In [ ]:
# GPU instances

instances = [
    'ml.p2.xlarge',
    'ml.p3.2xlarge'
]

The hyperparameters are passed as command line arguments to the training script. 

You can add/change them as you like. It's important to keep the `tracking_uri` and the `experiment_name` which are used by MLFlow.

The class `PyTorch` is part of the `SageMaker` python API. The parameters are important and you should probably not change most of them. The ones you may want to change are:

- `instance_type`, specify the instance you want
- `source_dir`, specify your script directory. Try to use global variable as much as possible

In [ ]:
from sagemaker.pytorch import PyTorch

from deep.constants import MLFLOW_SERVER, SAGEMAKER_ROLE

hyperparameters={
    'epochs': 5,
    'model_name': 'distilbert-base-uncased',
    'tracking_uri': MLFLOW_SERVER,
    'experiment_name': '1D-multihead-transformers',
    'iterative': False,
    'loss': 'focal'
}

estimator = PyTorch(
    entry_point='train.py',
    source_dir=str('../../../scripts/training/oguz/huggingface-multihead'),
    output_path=str(DEV_BUCKET / 'models/'),
    code_location=str(input_path),
    instance_type='ml.p3.2xlarge',
    instance_count=1,
    role=SAGEMAKER_ROLE,
    framework_version='1.8',
    py_version='py36',
    hyperparameters = hyperparameters,
    job_name=job_name,
)

In [ ]:
fit_arguments = {
    'train': str(input_path),
    'test': str(input_path)
}

In [ ]:
# Fit the estimator

estimator.fit(fit_arguments, job_name=job_name)

## Debugging (SageMaker JupyterLab)

In [ ]:
#!pip install cloudpathlib
#!pip install mlflow
#!pip install transformers

In [ ]:
from deep.constants import MLFLOW_SERVER, SAGEMAKER_ROLE

PATH = os.path.abspath('../../../../deep-experiments/scripts/training/oguz/huggingface-multihead/train.py')

In [ ]:
%env SM_OUTPUT_DATA_DIR=''
%env SM_MODEL_DIR=''
%env SM_NUM_GPUS=1
%env SM_CHANNEL_TRAIN={input_path}
%env SM_CHANNEL_TEST={input_path}

!python {PATH} --epochs 3 --tracking_uri {MLFLOW_SERVER} --experiment_name {'1D-multihead-transformers'} --model_name {'distilbert-base-uncased'}

## Evaluation

In [ ]:
import mlflow
import pandas as pd

from deep.constants import MLFLOW_SERVER

mlflow.set_tracking_uri(MLFLOW_SERVER)

experiment_name = "1D-multihead-transformers"
experiment = mlflow.get_experiment_by_name(experiment_name)
experiment_id = dict(experiment)['experiment_id']

runs = mlflow.search_runs(experiment_ids=experiment_id)
runs = runs.reindex(index=runs.index[::-1])
runs.head(10)

In [ ]:
import re

def get_cols(reg):
    cols = []
    matches = []
    for col in runs.columns:
        match = re.match(reg, col)
        if match:
            cols.append(col)
            matches.append('->'.join(match.groups()))
    return cols, matches

reg = 'metrics\.eval_(.+)-(.+)_binary_f1'
cols, matches = get_cols(reg)
print(cols)
print(matches)

### Subpillar Recall-Precision

In [ ]:
runs.sort_values('metrics.eval_subpillar_micro_f1', ascending=False).head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# get data
run_id = 9
data = runs.loc[run_id]

# get precision
reg = 'metrics\.eval_(.+)-(.+)_binary_precision'
cols, subpillars = get_cols(reg)
precision = data[cols]
precision = { k: v for k, v in zip(subpillars, precision.values ) }

# get recall
reg = 'metrics\.eval_(.+)-(.+)_binary_recall'
cols, subpillars = get_cols(reg)
recall = data[cols]
recall = { k: v for k, v in zip(subpillars, recall.values ) }

# get f1
reg = 'metrics\.eval_(.+)-(.+)_binary_f1'
cols, subpillars = get_cols(reg)
f1 = data[cols]
f1 = { k: v for k, v in zip(subpillars, f1.values ) }

# reorder
keys = list(f1.keys())
precision = [ precision[k] for k in keys ]
recall = [ recall[k] for k in keys ]
f1 = [ f1[k] for k in keys ]

# form a pandas dataframe
df = pd.DataFrame({
    'subpillar': keys,
    'precision': precision,
    'recall': recall,
    'f1': f1
})
df.head()

In [ ]:
# plot
sns.set_context('talk')
plt.figure(figsize=(12,12))
sns.scatterplot(
    data=df,
    x='precision',
    y='recall'
)
for i in range(df.shape[0]):
    plt.text(
        x=df.precision[i],
        y=df.recall[i]+0.01,
        s=df.subpillar[i],
        fontdict=dict(color='red', size=8, ha='center'),
        bbox=dict(facecolor='yellow', alpha=0.1))
plt.xlim(df.precision.min()-0.2, df.precision.max()+0.2)                
plt.ylim(df.recall.min()-0.2, df.recall.max()+0.2)              
plt.title('Subpillar precision-recall scatter')
plt.show()

In [ ]:
df.hist('f1', bins=25)
plt.show()

In [ ]:
# bad precision
df[df.precision < 0.5].head(100)

In [ ]:
# bad recall
df[df.recall < 0.5].head(100)

## Inference Preprocessing (Offline Testing Environment)

In [ ]:
import pandas as pd

DATA_PATH = 'leads.csv'
data = pd.read_csv(DATA_PATH)
data.head()

In [ ]:
from ast import literal_eval

lengths = {}

for field in ['extracted_text_as_paragraphs', 'extracted_text_as_sentences']:
    arr = data[field].apply(literal_eval).tolist()
    lengths[field] = [len(ds) for ds in arr]
    
    infer_df = pd.DataFrame.from_dict({
        'excerpt': [d for ds in arr for d in ds]
    })
    infer_df = infer_df[~(infer_df['excerpt'].str.len() == 0)]
    infer_df.to_csv(f'infer_{field}.csv', header=True, index=True)